<a href="https://colab.research.google.com/github/amirabbasgashtil/Linear-Programming/blob/main/simplex_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplex algorithm for optimizing a characteristic function